<a href="https://colab.research.google.com/github/YoonSungLee/Galaxies-Classification-By-Using-Deep-Learning/blob/master/Tutorial_and_Concept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 딥러닝으로 은하 분류하기

# 1번 문제 : Import

아래 과정은 다중 클래스 이미지 분류에 사용되는 레이어들을 불러오는 과정이다. 굵은 글씨로 표시된 Dense, Flatten, Conv2D, MaxPooling2D 레이어들이 각각 어떤 역할을 하는지 간단하게 적으시오.

* Dense : Fully Connected Layer로써, input_data에 가중치를 곱하고 편향을 더하여 output_data를 만들어낸다. 이 때 output_data의 feature의 수는 units으로 조절할 수 있고 다음 층으로 전달하기 전에 activation function을 정의할 수 도 있다. 해당 층에서 학습시켜야 할 파라미터의 수는 (input_data의 feature)*(weight)+(bias)이다.
* Flatten : 2차원 데이터를 1차원으로 펼쳐주는 역할을 한다.
* Conv2D : 기존의 Fully Connected Layer와는 달리 filter를 이용하여 학습 및 예측을 하는 층이다. filter는 input_data와의 합성곱으로 다음 층으로 전달하고 backpropagation을 통해 filter의 값들을 업데이트시킨다.
* MaxPooling2D : input_data의 특징을 더 잘 추출하는 단계로써, window의 크기에 따라 해당 크기 내의 가장 큰 값을 다음 layer로 전달해주는 역할을 한다.

* Dense: 딥러닝 및 머신러닝에 가장 기본적으로 이용되는 레이어
이며 입력층과 출력층을 모두 연결해준다.
* Conv2D: 영상 및 이미지 인식에 사용되는 레이어이며 그것들의 특징을 뽑아 새로운 2차원 데이터를 만들어낸다.
* MaxPooling2D: Conv2D 레이어의 출력 이미지에서 주요 값만 뽑아
크기가 작은 출력 영상을 만들어 지역적인 사소한 변화가 영향을
미치지 않도록 한다.
* Flatten: Conv2D와 MaxPooling2D 레이어를 거치면서 만들어진 2
차원 데이터를 Dense층에 연결시키기 위해 1차원 데이터로 만들어
주는 레이어

In [0]:
# 구글 드라이브 mount

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# 패키지 불러오기

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


# 2번 문제 : Image Data Generator

케라스에서는 모델에게 이미지 데이터를 쉽게 학습시킬 수 있도록 ImageDataGenerator 클래스를 제공한다. 아래 코드는 ImageDataGenerator 클래스를 이용해 특정 폴더에 분류된 이미지를 train, validation, test 데이터 셋으로 만드는 과정이다. A, B, C, D 에 들어갈 코드를 적으시오(Hint: A에 들어갈 코드는 호출된 함수이며, 함수의 인자로는 파일 경로, 이미지의 크기, 배치 크기, 분류 방식이 있다).

In [0]:
# 랜덤시드 고정
np.random.seed(3)

# 데이터 생성
# rescale: rescaling factor. Defaults to None.
# If None or 0, no rescaling is applied, otherwise we multiply the data by the value provided (before applying any other transformation).
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/gdrive/My Drive/Galaxies-Classification-By-Using-Deep-Learing/datasets/train_galaxy', # 파일 경로
    target_size=(50,50), # 이미지의 크기
    batch_size=3,# 배치 크기
    class_mode='categorical'# 분류 방식
)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    '/content/gdrive/My Drive/Galaxies-Classification-By-Using-Deep-Learing/datasets/val_galaxy', # 파일 경로
    target_size=(50,50), # 이미지의 크기
    batch_size=3,# 배치 크기
    class_mode='categorical'# 분류 방식
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/gdrive/My Drive/Galaxies-Classification-By-Using-Deep-Learing/datasets/test_galaxy', # 파일 경로
    target_size=(50,50), # 이미지의 크기
    batch_size=3,# 배치 크기
    class_mode='categorical'# 분류 방식
)

Found 10249 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


### flow_from_directory<br>

* 첫번재 인자 : 이미지 경로를 지정합니다.
* target_size : 패치 이미지 크기를 지정합니다. 폴더에 있는 원본 이미지 크기가 다르더라도 target_size에 지정된 크기로 자동 조절됩니다.
* batch_size : 배치 크기를 지정합니다.
* class_mode : 분류 방식에 대해서 지정합니다.
 * categorical : 2D one-hot 부호화된 라벨이 반환됩니다.
 * binary : 1D 이진 라벨이 반환됩니다.
 * sparse : 1D 정수 라벨이 반환됩니다.
 * None : 라벨이 반환되지 않습니다.

# 3번 문제 : CNN

모델 구성
<br>
<br>
model = Sequential()<br>
(......)<br>
(......)

위의 코드는 이미지 분류를 위한 컨볼루션 신경망(CNN) 모델을 구성하는 과정이다. 아래 조건에 맞도록 알맞은 코드를 적으시오.

조건<br>
* Conv2D 레이어 : 입력 이미지 크기 50\*50, 입력 이미지 채널 3개(RGB), 필터 크기 3\*3, 필터 수 32개, 활성화 함수 'relu'
* Conv2D 레이어 : 필터 크기 3\*3, 필터 수 64개, 활성화 함수 'relu'
* MaxPooling2D 레이어 : 풀 크기 2\*2
* Flatten 레이어
* Dense 레이어 : 출력 뉴런 수 128개, 활성화 함수 'relu'
* Dense 레이어 : 출력 뉴런 수 3개, 활성화 함수 'softmax'
* 단, padding과 stride는 우선 고려하지 않아도 좋다. 케라스는 이러한 조건들을 기본 값으로 설정해준다. 추후 별도로 학습하는 것을 추천한다. 학습이 되어있는 경우라면 자유롭게 구성하여도 좋다.

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(50,50,3)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 4번 문제 : Relu, Softmax

활성화 함수인 relu와 softmax가 무엇인지 간단히 설명하고 주로 언제 쓰이는지 서술하시오.

* relu : activation function 중의 하나로써 기존의 sigmoid function으로 인한 gradient vanishing problem을 해결하는 데 크게 기여한 함수이다. y = max(0,x) 꼴의 단순한 형태로써, 양수 값은 그대로 다음 layer로 흘려보내고 음수 값은 0의 값을 흘려보낸다. CNN 모델에 좋은 성능을 보이며 흔히 layer와 layer 사이의 activation function을 정의하는 데 사용한다.
* softmax : activation function 중의 하나로써 다중분류를 할 때 사용하는 함수이다. 여러 개의 출력값들을 exponential을 취한 뒤 전체 합을 각각의 exponential한 값에 나눠주면 된다. 이를 통해 출력값들의 합을 1로 만들어 정규화를 해준다. 이를 통해 확률적 접근이 가능하고, 정규화 전보다 각 값들의 차이를 더욱 분명하게 해 주는 효과가 있다.

* relu: 0보다 큰 입력 값에 대해선 1로 출력하고, 0보다 작은 입력 값
에 대해선 0으로 출력하는 활성화 함수. 기존에 쓰이던 sigmoid 활
성화 함수는 0~1사이의 값을 출력하기 때문에 학습을 하면 할수록
갱신되는 가중치가 0에 가까워져 학습이 잘 되지 않는 문제가 발생,
이를 해결하기 위해 나온 활성화 함수. 은닉층에 주로 사용된다.
* softmax: n개의 다른 이벤트들에 대해 n개의 확률 분포를 계산한
다. 출력 확률 범위는 0~1이며, 확률을 모두 합치면 1이 된다. 다중
클래스 분류 문제에서 출력층에 주로 쓰인다.

# 5번 문제 : Loss, Optimizer

In [0]:
# 모델 학습과정 설정

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

위의 코드는 모델의 학습과정을 설정하는 과정이다. 굵은 글씨로 표시된 loss, optimizer가 각각 무엇을 뜻하는지 간단히 설명하시요.

* loss : loss : 현재 가중치 세트를 평가하는 데 사용한 손실 함수 입니다. 다중 클래스 문제이므로 ‘categorical_crossentropy’으로 지정합니다.
* optimizer : 최적의 가중치를 검색하는 데 사용되는 최적화 알고리즘으로 효율적인 경사 하강법 알고리즘 중 하나인 ‘adam’을 사용합니다.
* metrics : 평가 척도를 나타내며 분류 문제에서는 일반적으로 ‘accuracy’으로 지정합니다.

* loss: 손실 함수를 설정하는 부분이다. 손실 함수는 딥러닝 모델이
학습을 하면서 실제값과 딥러닝의 학습 결과가 얼마나 차이 나는지
알려준다. 따라서 손실 함수의 결과 값이 작을수록 학습이 잘 되었
다고 할 수 있다.
* optimizer: 최적화 알고리즘을 설정하는 부분이다. 손실 함수의 결
과 값을 최소화 하는, 가장 최적의 가중치를 찾아낸다.

# 6번 문제 : Validation, Evaluate

아래 코드는 모델을 학습시키고 평가하여 정확도를 출력하는 과정이다. A, B, C, D, E에 들어갈 코드를 적으시오(단, 현재 epoch는 50으로 고정되어있다).

In [0]:
# 모델 학습시키기

hist = model.fit_generator(
    train_generator,
    steps_per_epoch=500, # 3416이 적절하지만 시간관계상 학습량을 줄였다.
    epochs=50,
    validation_data=val_generator,
    validation_steps=200
)

# 모델 평가
# 학습한 모델을 평가해봅니다. 제네레이터에서 제공되는 샘플로 평가할 때는 evaluate_generator 함수를 사용합니다.

print('--Evaluate--')
scores = model.evaluate_generator(test_generator, steps=200)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 사용하기
# 모델 사용 시에 제네레이터에서 제공되는 샘플을 입력할 때는 predict_generator 함수를 사용합니다.
# 예측 결과는 클래스별 확률 벡터로 출력되며, 클래스에 해당하는 열을 알기 위해서는 제네레이터의 ‘class_indices’를 출력하면 해당 열의 클래스명을 알려줍니다.
# print("-- Predict --")
# output = model.predict_generator(test_generator, steps=200)
# np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
# print(test_generator.class_indices)
# print(output)

Epoch 1/50
500/500 [==============================] - 569s 1s/step - loss: 0.8775 - acc: 0.6000 - val_loss: 0.6666 - val_acc: 0.7450
Epoch 2/50
500/500 [==============================] - 603s 1s/step - loss: 0.6900 - acc: 0.6967 - val_loss: 0.6720 - val_acc: 0.7500
Epoch 3/50
500/500 [==============================] - 623s 1s/step - loss: 0.6517 - acc: 0.7387 - val_loss: 0.6672 - val_acc: 0.7217
Epoch 4/50
500/500 [==============================] - 595s 1s/step - loss: 0.6279 - acc: 0.7513 - val_loss: 0.5273 - val_acc: 0.7900
Epoch 5/50
500/500 [==============================] - 602s 1s/step - loss: 0.5795 - acc: 0.7627 - val_loss: 0.5089 - val_acc: 0.8083
Epoch 6/50
500/500 [==============================] - 595s 1s/step - loss: 0.5717 - acc: 0.7587 - val_loss: 0.5241 - val_acc: 0.7783
Epoch 7/50
500/500 [==============================] - 517s 1s/step - loss: 0.5707 - acc: 0.7693 - val_loss: 0.5208 - val_acc: 0.7917
Epoch 8/50
500/500 [==============================] - 5s 9ms/step - l

In [0]:
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

acc: 76.50%


케라스에서는 모델을 학습시킬 때 주로 fit() 함수를 사용하지만 제네레이터로 생성된 배치로 학습시킬 경우에는 fit_generator() 함수를 사용합니다. 본 예제에서는 ImageDataGenerator라는 제네레이터로 이미지를 담고 있는 배치로 학습시키기 때문에 fit_generator() 함수를 사용하겠습니다.

* 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 train_generator으로 지정합니다.
* steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정합니다. 예를 들어 총 45개의 훈련 샘플이 있고 배치사이즈가 3이면 15 스텝으로 지정할 수 있습니다.
* epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정합니다.
* validation_data : 검증데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 validation_generator으로 지정합니다.
* validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정합니다. 예를 들어 총 15개의 검증 샘플이 있고 배치사이즈가 3이면 5 스텝으로 지정할 수 있습니다.

# 7번 문제 : Accuracy

은하를 분류함에 있어서 RGB 조건이 반드시 반영되어야 하는 조건일까? 다음과 같이, 위의 코드는 예시일 뿐이다. 이러한 부분을 고려하여 더 높은 훈련 정확도를 갖는 모델을 설계하시오(단, 논리적인 전개를 함께 서술하시오).

처음 모델을 설계할 떄에는 주어진 이미지가 RGB 이미지이기때문에 의심의 여지 없이 해당 데이터로 학습을 시켰다. 하지만 정말로 RGB 조건이 필요한지에 대한 의문은 기존의 고정관념을 깨는 질문이었다. train 데이터의 edge, smooth, spiral 이미지를 살펴보면 형태는 다르지만 컬러색이 굉장히 유사하다는 것을 확인할 수 있다. 즉, RGB 정보가 이들을 분류하기에 큰 도움이 되지 않는 정보일 수 있다고 예측해볼 수 있다. 따라서 RGB가 아닌 Gray Scale이미지를 통해 비슷한 성능을 내는지 프로토타입을 통해 확인하고 적절한 방법을 선택하는 것이 중요하다.

# 8번 문제 : Another Model

은하를 분류하는 딥러닝 모델을 만듦에 있어 CNN을 이용할 필요도 없다. 개선된 다른 모델 사용 등의 방법을 고려하여, 더 높은 훈련 정확도를 갖는 모델을 설계하시오(단, 논리적인 전개를 함께 서술하시오).

이미지 데이터 분류는 일반적으로 CNN이 사용된다. 이 외에도 다른 머신러닝 기법을 사용할 수 있다. RandomForestClassifier, ExtraTreeClassifier, AdaBoostClassifier, XGBoostClassifier, CatBoostClassifier, XGBClassifier, LGBMClassifier 등 많은 분류모델들이 있다. 이러한 머신러닝 모델을 사용할 때의 주의점이 있는데, 해당 데이터는 이미지 데이터이므로 feature의 수가 너무 많아 그대로 학습시키면 성능이 저하되는 문제가 발생한다. 따라서 PCA기법 등을 이용하여 차원을 축소시킨 다음에(물론 축소된 차원들은 기존 데이터의 분산을 잘 설명할 수 있어야 할 것이다) 모델을 학습시키는 것이 하나의 방법이 될 수 있을 것이다.

# 9번 문제 : Conclusion

1~8번 문제 해결과정을 종합적으로 고려하여, 프로젝트의 결론을 서술하시오. 또한 이 프로젝트 경험을 기반으로 삼아 추후 어떤 학습 방향을 설정해야 할지 스스로 판단하시오.

이번 프로젝트는 기존에 알고 있던 CNN 모델을 다시 한 번 모델링해봄으로써 그 개념을 확실하게 하는 기회를 제공해주었다. 이번 프로젝트를 통한 피드백은 다음과 같다.

* 데이터 추출 방법을 생각해봐야 할 것이다. keras의 dataset에서 제공해주는 이미지 데이터만 학습하다가 실제 데이터를 받아서 학습하는 과정에서 중대한 문제점을 발견했다. 구글 코랩을 사용하는 입장에서 구글 드라이브에 이미지를 올리는 데 시간이 너무 오래 걸린다는 점이다. 이번 데이터는 단순히 기다리며 해결했지만 항상 마냥 기다릴 수 만은 없을 것이다. 이를 해결하기 위한 방법은 이미지를 numpy matrix화시켜서 저장한 다음에 올리는 방법이 있고, 구글 코랩 대신 다른 방법(로컬 GPU 구하기, AWS 등)을 이용하여 로컬 이미지를 바로 학습시키는 방법이 있을 것이다. 이에 대해서 더욱 찾아봐야 할 것이다.
* 학습시간의 문제이다. 앞의 문제와 비슷한 문제인데, 이미지 데이터는 학습시간 또한 굉장히 오래 걸린다. 이번 프로젝트의 경우에는 1 epoch당 모든 이미지를 학습해보지도 못한 채 다음 epoch으로 넘어가도록 설계할 수 밖에 없었다. 구글 코랩의 한계가 드러나는 시점이다. 따라서 로컬 GPU를 구비하거나 AWS를 사용하는 등 GPU 확보에 대해서 생각해보아야 할 것이다.
* 이 외의 CNN 기법들에 대해서는 꾸준히 배우고 있고 이는 탐구를 통해 충분히 해결할 수 있는 문제이다. 이보다는 작업환경에 대한 문제를 해결할 방법을 모색해야 할 것이다.